# Create Historical DTA Major Version

Creates DTA Major versions directly from silver data for historical DTAs.

**Purpose:**
Historical tsDTAs are already-executed DTAs that skip the draft phase and go directly to DTA Major status.

**Can be called from:**
1. `job_cdm_version_manager` with `action=CREATE_DTA_MAJOR` (single dta_id via widget)
2. `job_cdm_dta_create` via run_job_task (dta_ids via task values)

**What it does:**
1. Gets DTA ID(s) from widget parameter OR task values
2. For each DTA, reads silver data and builds gold-level aggregation
3. Creates DTA Major version directly in library table
4. Registers version in md_version_registry
5. Updates DTA entity with current_version_tag


In [ ]:
# Cell 1: Imports
import json
import uuid
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, BooleanType, ArrayType, LongType

print("="*80)
print("CREATE HISTORICAL DTA MAJOR VERSION")
print("="*80)


In [ ]:
# Cell 2: Get Configuration from Setup Task and Widget Parameters
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
versioning_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="versioning"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

catalog = globals_dict['catalog']
silver_schema = globals_dict['silver_schema']
gold_schema = globals_dict['gold_schema']

# Widget parameters (from job_cdm_version_manager)
dbutils.widgets.text("dta_id", "", "DTA ID (single)")
dbutils.widgets.text("library_type", "", "Library Type (defaults from config)")

dta_id_param = dbutils.widgets.get("dta_id")
library_type_param = dbutils.widgets.get("library_type")

# Determine DTA IDs: from widget (single) OR from task values (multiple)
if dta_id_param:
    # Called from job_cdm_version_manager with single dta_id
    created_dta_ids = [dta_id_param]
    created_dta_numbers = []  # Will be looked up from DTA table
    print(f"Mode: Single DTA from widget parameter")
else:
    # Called from job_cdm_dta_create via task values
    try:
        created_dta_ids = json.loads(dbutils.jobs.taskValues.get(taskKey="create_dta_instance", key="created_dta_ids"))
        created_dta_numbers = json.loads(dbutils.jobs.taskValues.get(taskKey="create_dta_instance", key="created_dta_numbers"))
        print(f"Mode: Multiple DTAs from create_dta_instance task values")
    except Exception as e:
        raise ValueError(f"No dta_id provided via widget and no task values found: {e}")

# Determine library_type: from widget, task values, or config
if library_type_param:
    library_type = library_type_param
else:
    try:
        library_type = dbutils.jobs.taskValues.get(taskKey="create_dta_instance", key="library_type")
    except:
        library_tables = versioning_dict.get('library_tables', [])
        library_type = library_tables[0].get('library_type', 'transfer_variables') if library_tables else 'transfer_variables'

# Build library table and registry table names from config
library_tables = versioning_dict.get('library_tables', [])
registry_table_name = versioning_dict.get('registry_table', 'md_version_registry')

library_config = next((t for t in library_tables if t.get('library_type') == library_type), None)
library_table_name = library_config.get('name', 'md_transfer_variables_library') if library_config else 'md_transfer_variables_library'

library_table = f"{catalog}.{gold_schema}.{library_table_name}"
registry_table = f"{catalog}.{gold_schema}.{registry_table_name}"

print(f"Catalog: {catalog}")
print(f"Silver Schema: {silver_schema}")
print(f"Gold Schema: {gold_schema}")
print(f"Library Type: {library_type}")
print(f"DTAs to process: {len(created_dta_ids)}")


In [ ]:
# Cell 3: Validate Configuration
print(f"\n{'='*60}")
print(f"Configuration Summary")
print(f"{'='*60}")
print(f"Library Type: {library_type}")
print(f"Library Table: {library_table}")
print(f"Registry Table: {registry_table}")
print(f"DTA IDs: {created_dta_ids}")


In [ ]:
# Cell 4: Configure Additional Table Names
silver_table = f"{catalog}.{silver_schema}.md_transfer_variable_field_normalized"
dta_table = f"{catalog}.{gold_schema}.dta"

print(f"\nSilver Source: {silver_table}")
print(f"DTA Table: {dta_table}")


In [ ]:
# Cell 5: Get Current Library Version
from clinical_data_standards_framework.versioning import (
    get_version_registry_schema,
    initialize_library_version
)

# Check if library table exists and has data
if not spark.catalog.tableExists(library_table):
    print(f"Library table does not exist. Will be created.")
    base_library_version = "1.0"
elif spark.table(library_table).count() == 0:
    print(f"Library table is empty. Will initialize with version 1.0")
    base_library_version = "1.0"
else:
    # Get latest library major version
    df_registry = spark.table(registry_table).filter(
        (F.col("library_type") == library_type) &
        (F.col("version_type") == "LIBRARY_MAJOR") &
        (F.col("status") == "ACTIVE")
    )
    if df_registry.count() > 0:
        base_library_version = df_registry.orderBy(F.desc("created_ts")).first()["version_tag"]
    else:
        base_library_version = "1.0"

print(f"Base library version: {base_library_version}")


In [ ]:
# Cell 6: Read DTA Metadata (including dta_number for version tags)
df_dta = spark.table(dta_table).filter(F.col("dta_id").isin(created_dta_ids))
dta_metadata = {row["dta_id"]: row.asDict() for row in df_dta.collect()}

# If dta_numbers weren't provided via task values, build from DTA table
if not created_dta_numbers:
    created_dta_numbers = [dta_metadata[dta_id]["dta_number"] for dta_id in created_dta_ids]
    print(f"Looked up DTA numbers from table: {created_dta_numbers}")

print(f"\nLoaded metadata for {len(dta_metadata)} DTAs")


In [ ]:
# Cell 7: Process Each DTA and Create DTA Major Version
now = datetime.now()

print(f"\n{'='*80}")
print("Processing DTAs and creating DTA Major versions...")
print(f"{'='*80}")

version_results = {}

for idx, dta_id in enumerate(created_dta_ids):
    meta = dta_metadata.get(dta_id, {})
    dta_number = meta.get("dta_number") or created_dta_numbers[idx]  # Get dta_number from metadata or list
    trial_id = meta.get("trial_id")
    stream_type = meta.get("data_stream_type")
    provider = meta.get("data_provider_name")
    
    print(f"\nProcessing DTA: {dta_number} (ID: {dta_id[:8]}...)")
    print(f"  Trial: {trial_id}, Stream: {stream_type}, Provider: {provider}")
    
    # Filter silver data for this DTA context
    df_silver = spark.table(silver_table).filter(
        (F.col("trial_id") == trial_id) &
        (F.col("data_stream_type") == stream_type) &
        (F.col("data_provider_name") == provider) &
        (F.col("row_status").isin("COMPLETED", "MANUAL_REVIEW_REQUIRED")) &
        (F.col("definition_hash").isNotNull())
    )
    
    record_count = df_silver.count()
    print(f"  Records: {record_count}")
    
    if record_count == 0:
        print(f"  Skipping - no records to process")
        continue
    
    # Create version tag for DTA Major using sequential DTA number
    # Format: {library_major}-{dta_number}-v{major}.{minor}
    # Example: 1.0-DTA001-v1.0
    version_tag = f"{base_library_version}-{dta_number}-v1.0"
    print(f"  Version tag: {version_tag}")
    
    # Aggregate silver data to gold format (similar to build_gold_transfer_variables)
    # Note: Silver table uses 'format' column, not 'data_type_format'
    df_gold = df_silver.groupBy("definition_hash").agg(
        F.first("transfer_variable_name").alias("transfer_variable_name"),
        F.first("transfer_variable_label").alias("transfer_variable_label"),
        F.first("format").alias("format"),  # Column is 'format' in silver table
        F.first("anticipated_max_length").alias("anticipated_max_length"),
        F.first("variable_description").alias("variable_description"),
        F.first("transfer_file_key").alias("transfer_file_key"),
        F.first("populate_for_all_records").alias("populate_for_all_records"),
        F.first("codelist_values").alias("codelist_values"),
        F.count("*").alias("usage_count"),
        F.sum(F.when(F.col("row_status") == "COMPLETED", 1).otherwise(0)).alias("completed_count"),
        F.sum(F.when(F.col("row_status") == "MANUAL_REVIEW_REQUIRED", 1).otherwise(0)).alias("review_required_count"),
        F.collect_set(
            F.struct("trial_id", "data_stream_type", "data_provider_name", "row_status")
        ).alias("used_in_trials")
    ).withColumn(
        "needs_review",
        F.col("review_required_count") > 0
    )
    
    # Add versioning columns
    df_gold = df_gold.withColumn("library_version", F.lit(version_tag)) \
        .withColumn("is_major_version", F.lit(False)) \
        .withColumn("is_dta_major", F.lit(True)) \
        .withColumn("parent_version", F.lit(base_library_version)) \
        .withColumn("effective_start_ts", F.lit(now)) \
        .withColumn("effective_end_ts", F.lit(None).cast("timestamp")) \
        .withColumn("is_current", F.lit(True))
    
    # Add audit columns
    df_gold = df_gold.withColumn("created_by_principal", F.lit(created_by_principal)) \
        .withColumn("created_ts", F.lit(now)) \
        .withColumn("last_updated_by_principal", F.lit(created_by_principal)) \
        .withColumn("last_updated_ts", F.lit(now)) \
        .withColumn("databricks_job_id", F.lit(databricks_job_id)) \
        .withColumn("databricks_job_name", F.lit(databricks_job_name)) \
        .withColumn("databricks_run_id", F.lit(databricks_run_id))
    
    # Write to library table
    df_gold.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable(library_table)
    print(f"  Wrote {df_gold.count()} records to library table")
    
    version_results[dta_id] = version_tag


In [ ]:
# Cell 8: Register Versions in Registry
print(f"\n{'='*80}")
print("Registering versions in md_version_registry...")
print(f"{'='*80}")

# Ensure registry table exists
if not spark.catalog.tableExists(registry_table):
    print(f"Creating registry table: {registry_table}")
    empty_df = spark.createDataFrame([], schema=get_version_registry_schema())
    empty_df.write.format("delta").mode("overwrite").saveAsTable(registry_table)

registry_records = []
for dta_id, version_tag in version_results.items():
    meta = dta_metadata.get(dta_id, {})
    record_count = spark.table(library_table).filter(F.col("library_version") == version_tag).count()
    
    registry_record = {
        "version_tag": version_tag,
        "library_type": library_type,
        "version_type": "DTA_MAJOR",
        "is_latest_major": False,  # Not a library major
        "dta_id": dta_id,
        "dta_trial_id": meta.get("trial_id"),
        "dta_stream_type": meta.get("data_stream_type"),
        "dta_provider_name": meta.get("data_provider_name"),
        "parent_version": base_library_version,
        "record_count": record_count,
        "status": "ACTIVE",
        "notes": "Historical DTA - auto-imported",
        "created_by_principal": created_by_principal,
        "created_ts": now,
        "last_updated_by_principal": created_by_principal,
        "last_updated_ts": now,
        "databricks_job_id": databricks_job_id,
        "databricks_job_name": databricks_job_name,
        "databricks_run_id": databricks_run_id
    }
    registry_records.append(registry_record)
    print(f"  Registered: {version_tag} ({record_count} records)")

if registry_records:
    registry_df = spark.createDataFrame(registry_records, schema=get_version_registry_schema())
    registry_df.write.format("delta").mode("append").saveAsTable(registry_table)
    print(f"\nRegistered {len(registry_records)} version(s)")


In [ ]:
# Cell 9: Update DTA Entities with Version Tags
print(f"\n{'='*80}")
print("Updating DTA entities with version tags...")
print(f"{'='*80}")

for dta_id, version_tag in version_results.items():
    spark.sql(f"""
        UPDATE {dta_table}
        SET current_version_tag = '{version_tag}',
            last_updated_by_principal = '{created_by_principal}',
            last_updated_ts = current_timestamp(),
            databricks_job_id = '{databricks_job_id}',
            databricks_run_id = '{databricks_run_id}'
        WHERE dta_id = '{dta_id}'
    """)
    print(f"  Updated DTA {dta_id} with version {version_tag}")


In [ ]:
# Cell 10: Summary
print(f"\n{'='*80}")
print("Summary")
print(f"{'='*80}")
print(f"DTAs processed: {len(version_results)}")
for dta_id, version_tag in version_results.items():
    meta = dta_metadata.get(dta_id, {})
    print(f"  {dta_id[:8]}... -> {version_tag}")
    print(f"    Trial: {meta.get('trial_id')}, Status: {meta.get('status')}")
